
# Week 1 Lab – Part 2: Multiple Features and Polynomial Regression (House Prices)

In this second part of the lab we will extend linear regression from **one feature** to **multiple features**, and then to **polynomial regression**, using **house prices** as a running example.

We keep the same notation as in class:

- The $i$-th training example is $(x^{(i)}, y^{(i)})$.
- Each input $x^{(i)}$ has $n$ features: 
  $$
  x^{(i)} = \big[x_1^{(i)}, x_2^{(i)}, \dots, x_n^{(i)}\big].
  $$
- The model (hypothesis) is a function $f_{w,b}(x)$ that depends on parameters $w$ and $b$:
  $$
  f_{w,b}(x) = w_1 x_1 + w_2 x_2 + \dots + w_n x_n + b.
  $$


## 0.1 Linear Regression with Multiple Features

For one feature we had
$$
f_{w,b}(x) = wx + b.
$$

For **multiple features**, with $\vec{x} \in \mathbb{R}^n$ and $\vec{w} \in \mathbb{R}^n$, we write
$$
f_{\vec{w}, b}(\vec{x}) = \sum_{j=1}^{n} w_j x_j + b.
$$

In **vector notation** this becomes
$$
f_{\vec{w}, b}(\vec{x}) = \vec{w}^T \vec{x} + b.
$$

For a whole dataset with $m$ examples, we collect the inputs into a **design matrix** $\mathbf{X} \in \mathbb{R}^{m \times n}$:
- Row $i$ of $\mathbf{X}$ is $(x_1^{(i)}, \dots, x_n^{(i)})$.
- The target vector is $\vec{y} \in \mathbb{R}^m$, with entries $y^{(i)}$.

Then the predictions for all examples at once are
$$
\hat{\vec{y}} = \mathbf{X} \vec{w} + b \,\vec{1},
$$
where $\vec{1}$ is the vector of all ones (size $m$).


## 0.2 Cost Function for Multiple Features

The cost function is the same **mean squared error** we used before, now written for the multivariate case:

$$
J(\vec{w}, b) = \frac{1}{2m} \sum_{i=1}^{m} \left( f_{\vec{w}, b}(\vec{x}^{(i)}) - y^{(i)} \right)^2.
$$


## 0.3 Gradient Descent for Multiple Features

The partial derivatives of $J(\vec{w}, b)$ are:

- For each $w_j$:
  $$
  \frac{\partial J(\vec{w}, b)}{\partial w_j} 
  = \frac{1}{m} \sum_{i=1}^{m} \big( f_{\vec{w}, b}(\vec{x}^{(i)}) - y^{(i)} \big)\, x_j^{(i)}.
  $$

- For the bias term $b$:
  $$
  \frac{\partial J(\vec{w}, b)}{\partial b} 
  = \frac{1}{m} \sum_{i=1}^{m} \big( f_{\vec{w}, b}(\vec{x}^{(i)}) - y^{(i)} \big).
  $$

In **vector/matrix form**, let
- $\hat{\vec{y}} = f_{\vec{w}, b}(\mathbf{X})$ (the vector of predictions),
- $\vec{e} = \hat{\vec{y}} - \vec{y}$ (the error vector).

Then:
$$
\nabla_{\vec{w}} J(\vec{w}, b) 
= \frac{1}{m} \,\mathbf{X}^T \vec{e},
\qquad
\frac{\partial J(\vec{w}, b)}{\partial b}
= \frac{1}{m} \sum_{i=1}^{m} e^{(i)}.
$$

The gradient descent updates are:
$$
\vec{w} := \vec{w} - \alpha \, \nabla_{\vec{w}} J(\vec{w}, b),
\qquad
b := b - \alpha \, \frac{\partial J(\vec{w}, b)}{\partial b},
$$
where $\alpha$ is the learning rate.


## 1. Setup

In [ ]:

# Install required libraries (run this once if needed)
%pip install numpy pandas matplotlib


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.set_printoptions(precision=4, suppress=True)



## 2. A Simple House Prices Dataset

We will simulate a small house-price dataset with the following features:

- $x_1$ = `size_m2` (size of the house in square meters),
- $x_2$ = `bedrooms` (number of bedrooms),
- $x_3$ = `front_m` (front of the lot in meters),
- $x_4$ = `depth_m` (depth of the lot in meters).

The target $y$ is the house price (in thousands of dollars).

We will create data that roughly follows a linear relationship plus some noise.


In [ ]:

# Number of examples
m = 80

rng = np.random.default_rng(0)

# Features
size_m2 = rng.uniform(50, 200, m)        # 50 to 200 m^2
bedrooms = rng.integers(1, 5, m)         # 1 to 4 bedrooms
front_m = rng.uniform(5, 15, m)          # 5 to 15 m
depth_m = rng.uniform(8, 25, m)          # 8 to 25 m

# "True" underlying parameters (unknown to the algorithm)
true_w = np.array([1.5, 10.0, 3.0, 2.0])  # impact of each feature on price
true_b = 50.0                             # base price in thousands
noise = rng.normal(0, 10, m)              # noise in thousands

# Build design matrix X and target y
X = np.column_stack([size_m2, bedrooms, front_m, depth_m])
y = X @ true_w + true_b + noise

# Put into a DataFrame just to inspect
data = pd.DataFrame({
    "size_m2": size_m2,
    "bedrooms": bedrooms,
    "front_m": front_m,
    "depth_m": depth_m,
    "price_k": y,
})

data.head()


### 2.1 Visualize Size vs Price

In [ ]:

plt.figure()
plt.scatter(data["size_m2"], data["price_k"])
plt.xlabel("Size (m^2)")
plt.ylabel("Price (thousands)")
plt.title("House size vs price")
plt.show()



## 3. Vectorized Model Implementation

We now implement the model and cost function using **NumPy arrays** and **vectorized operations**.

### 3.1 Hypothesis $f_{w,b}(x)$

For a dataset with matrix $X$ and parameters $w$ and $b$, the vector of predictions is:
$$
\hat{y} = X w + b.
$$


In [ ]:

def predict(X, w, b):
    """Compute predictions f_{w,b}(x) for all examples.

    Parameters
    ----------
    X : np.ndarray, shape (m, n)
        Design matrix: each row is x^(i).
    w : np.ndarray, shape (n,)
        Parameter vector.
    b : float
        Bias term.

    Returns
    -------
    y_hat : np.ndarray, shape (m,)
        Vector of predictions for each example.
    """
    return X @ w + b  # vectorized: matrix-vector product + scalar



### 3.2 Cost Function $J(w,b)$ (Vectorized)

We recall
$$
J(w,b) = \frac{1}{2m} \sum_{i=1}^{m} \big( f_{w,b}(x^{(i)}) - y^{(i)} \big)^2.
$$

In vector form, if we define the error vector $e = \hat{y} - y$, then
$$
J(w,b) = \frac{1}{2m} e^T e.
$$


In [ ]:

def compute_cost(X, y, w, b):
    """Compute the cost J(w,b) for linear regression with multiple features.

    Uses the vectorized formula:
        J = (1 / (2m)) * (y_hat - y)^T (y_hat - y)

    Parameters
    ----------
    X : np.ndarray, shape (m, n)
    y : np.ndarray, shape (m,)
    w : np.ndarray, shape (n,)
    b : float

    Returns
    -------
    cost : float
    """
    m = X.shape[0]
    y_hat = predict(X, w, b)
    error = y_hat - y
    cost = (error @ error) / (2 * m)
    return cost

# Test with w = 0, b = 0
n = X.shape[1]
w_test = np.zeros(n)
b_test = 0.0
print("Cost with w=0, b=0:", compute_cost(X, y, w_test, b_test))



## 4. Gradient of the Cost Function (Vectorized)

We derived:
$$
\nabla_w J(w,b) = \frac{1}{m} X^T (\hat{y} - y), \qquad
\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)}).
$$

We now implement this directly in NumPy.


In [ ]:

def compute_gradient(X, y, w, b):
    """Compute the gradients of J with respect to w and b.

    Vectorized formulas:
        dj_dw = (1/m) * X^T (y_hat - y)
        dj_db = (1/m) * sum(y_hat - y)
    """
    m = X.shape[0]
    y_hat = predict(X, w, b)
    error = y_hat - y

    dj_dw = (X.T @ error) / m
    dj_db = np.sum(error) / m
    return dj_dw, dj_db

dj_dw_test, dj_db_test = compute_gradient(X, y, w_test, b_test)
print("Gradient at w=0, b=0:")
print("dj_dw:", dj_dw_test)
print("dj_db:", dj_db_test)



## 5. Gradient Descent for Multiple Features

We now put everything together into a gradient descent loop.

At each iteration:
1. Compute $\hat{y} = f_{w,b}(X)$.
2. Compute the gradients $\nabla_w J$ and $\partial J / \partial b$.
3. Update
   $$
   w := w - \alpha \nabla_w J, \qquad
   b := b - \alpha \frac{\partial J}{\partial b}.
   $$


In [ ]:

def gradient_descent(X, y, w_init, b_init, alpha, num_iterations):
    """Run gradient descent to learn w and b.

    Parameters
    ----------
    X : np.ndarray, shape (m, n)
    y : np.ndarray, shape (m,)
    w_init : np.ndarray, shape (n,)
    b_init : float
    alpha : float
        Learning rate.
    num_iterations : int

    Returns
    -------
    w : np.ndarray, shape (n,)
    b : float
    history_it : list of int
    history_cost : list of float
    """
    w = w_init.copy()
    b = b_init
    history_it = []
    history_cost = []

    for i in range(num_iterations):
        dj_dw, dj_db = compute_gradient(X, y, w, b)
        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        if i % 10 == 0 or i == num_iterations - 1:
            cost = compute_cost(X, y, w, b)
            history_it.append(i)
            history_cost.append(cost)
            print(f"Iteration {i:4d}: cost = {cost:8.4f}")

    return w, b, history_it, history_cost

alpha = 1e-4
num_iterations = 1000
w_init = np.zeros(n)
b_init = 0.0

w_learned, b_learned, it_hist, cost_hist = gradient_descent(X, y, w_init, b_init, alpha, num_iterations)
print("\nLearned parameters:")
print("w =", w_learned)
print("b =", b_learned)


### 5.1 Plot the Cost over Iterations

In [ ]:

plt.figure()
plt.plot(it_hist[15:], cost_hist[15:])
plt.xlabel("Iteration")
plt.ylabel("Cost J(w,b)")
plt.title("Gradient Descent: Cost vs Iterations (multiple features)")
plt.show()

plt.figure()
plt.semilogy(it_hist, cost_hist)
plt.xlabel("Iteration")
plt.ylabel("Cost J(w,b) (log scale)")
plt.title("Gradient Descent: Cost vs Iterations (log scale)")
plt.show()


### 5.2 Compare Predicted vs True Prices

In [ ]:

y_pred = predict(X, w_learned, b_learned)

plt.figure()
plt.scatter(y, y_pred)
plt.xlabel("True price (thousands)")
plt.ylabel("Predicted price (thousands)")
plt.title("Predicted vs True Prices")
plt.plot([y.min(), y.max()], [y.min(), y.max()])  # diagonal line
plt.show()



## 6. Feature Scaling

Gradient descent can be **slow** if the features have very different scales.

For example, in our dataset:
- `size_m2` might be around 100–200,
- `bedrooms` is around 1–4,
- `front_m` and `depth_m` have yet other ranges.

A common solution is to **scale** each feature, for example using **standardization**:
$$
x_j^{(i)} \leftarrow \frac{x_j^{(i)} - \mu_j}{\sigma_j},
$$
where $\mu_j$ is the mean of feature $j$, and $\sigma_j$ its standard deviation.

We will implement a simple feature scaling function and compare convergence **with** and **without** scaling.


In [ ]:

def feature_scale(X):
    """Standardize each feature to have mean 0 and standard deviation 1.

    Returns
    -------
    X_scaled : np.ndarray, shape (m, n)
    means : np.ndarray, shape (n,)
    stds : np.ndarray, shape (n,)
    """
    means = X.mean(axis=0)
    stds = X.std(axis=0, ddof=0)
    X_scaled = (X - means) / stds
    return X_scaled, means, stds

X_scaled, X_means, X_stds = feature_scale(X)
print("Feature means:", X_means)
print("Feature stds:", X_stds)


### 6.1 Gradient Descent with and without Scaling

In [ ]:

# Without scaling
alpha_no_scale = 1e-4
w0 = np.zeros(n)
b0 = 0.0
w_ns, b_ns, it_ns, cost_ns = gradient_descent(X, y, w0, b0, alpha_no_scale, 300)

# With scaling
alpha_scale = 1e-1  # we can use a larger learning rate after scaling
w0_s = np.zeros(n)
b0_s = 0.0
w_s, b_s, it_s, cost_s = gradient_descent(X_scaled, y, w0_s, b0_s, alpha_scale, 300)

plt.figure()
plt.plot(it_ns[2:], cost_ns[2:], label="No scaling")
plt.plot(it_s[2:], cost_s[2:], label="With scaling")
plt.xlabel("Iteration")
plt.ylabel("Cost J(w,b)")
plt.title("Effect of Feature Scaling on Convergence")
plt.legend()
plt.show()



## 7. Feature Engineering

Sometimes we can create **new features** that better capture the structure of the problem.

Example for houses:
- We know that the *area* of the lot is roughly `front_m × depth_m`.
- We can create a new feature:
  $$
  \text{area} = \text{front\_m} \times \text{depth\_m}.
  $$

We will add this `area` feature to our design matrix and see its effect.


In [ ]:

area = data["front_m"].to_numpy() * data["depth_m"].to_numpy()
data["area_m2"] = area

# New design matrix with the added engineered feature
X_fe = np.column_stack([
    data["size_m2"].to_numpy(),
    data["bedrooms"].to_numpy(),
    data["front_m"].to_numpy(),
    data["depth_m"].to_numpy(),
    data["area_m2"].to_numpy(),
])

n_fe = X_fe.shape[1]
X_fe_scaled, means_fe, stds_fe = feature_scale(X_fe)

w0_fe = np.zeros(n_fe)
b0_fe = 0.0
alpha_fe = 1e-1

w_fe, b_fe, it_fe, cost_fe = gradient_descent(X_fe_scaled, y, w0_fe, b0_fe, alpha_fe, 300)

print("Learned parameters with engineered area feature:")
print("w_fe =", w_fe)
print("b_fe =", b_fe)



## 8. Polynomial Regression

Linear regression can also model **non-linear relationships** by adding **polynomial features**.

Example: if price depends non-linearly on size, we can add $x_{\text{size}}^2$ as a new feature:
$$
f_{w,b}(x) = w_1 \,\text{size} + w_2 \,\text{size}^2 + b.
$$

This is still *linear in the parameters* $w_1, w_2, b$, so we can use the same algorithms.

We will:
1. Take just the `size_m2` feature.
2. Create a second feature `size_m2^2`.
3. Fit a polynomial regression model and visualize the curve.


In [ ]:

size_only = data["size_m2"].to_numpy()
y_price = data["price_k"].to_numpy()

X_poly = np.column_stack([size_only, size_only ** 2])
X_poly_scaled, means_poly, stds_poly = feature_scale(X_poly)

n_poly = X_poly_scaled.shape[1]
w0_poly = np.zeros(n_poly)
b0_poly = 0.0
alpha_poly = 1e-1

w_poly, b_poly, it_poly, cost_poly = gradient_descent(X_poly_scaled, y_price, w0_poly, b0_poly, alpha_poly, 500)

print("Learned parameters for polynomial regression (size and size^2):")
print("w_poly =", w_poly)
print("b_poly =", b_poly)


### 8.1 Visualize Polynomial Fit

In [ ]:

# Create a smooth grid of sizes for plotting the curve
size_grid = np.linspace(size_only.min(), size_only.max(), 200)
X_grid = np.column_stack([size_grid, size_grid ** 2])

# Apply same scaling as training
X_grid_scaled = (X_grid - means_poly) / stds_poly
y_grid_pred = predict(X_grid_scaled, w_poly, b_poly)

plt.figure()
plt.scatter(size_only, y_price, label="Data")
plt.plot(size_grid, y_grid_pred, label="Polynomial fit")
plt.xlabel("Size (m^2)")
plt.ylabel("Price (thousands)")
plt.title("Polynomial Regression: Price vs Size")
plt.legend()
plt.show()



## 9. Exercises (for You to Try)

1. **Change the learning rate $\alpha$** for the multifeature model:
   - Try values like `1e-3`, `5e-4`, `5e-2` (with scaling).
   - How does the convergence speed change? Does it diverge for some values?

2. **Try different engineered features**:
   - For example, try `size_per_bedroom = size_m2 / bedrooms` (be careful with division by zero).
   - Add it as an extra column and see if the cost decreases faster or to a smaller value.

3. **Compare models with and without the `area` feature**:
   - Train two models: one with features `[size, bedrooms, front, depth]` and one with `[size, bedrooms, front, depth, area]`.
   - Compare training cost and the scatter plot of true vs predicted prices.

4. **Increase the polynomial degree**:
   - Add a `size_m2^3` column and repeat the polynomial regression part.
   - Does the fit improve? Does it start to look like it is overfitting the noise?

5. **(Optional) Use a real dataset**:
   - If you have a CSV file with real house data, load it with `pandas.read_csv`,
   - Build the design matrix $X$ and target $y$,
   - Apply the same steps: scaling, gradient descent, feature engineering, and polynomial regression.
